In [1]:
#s3_prediction = "s3://aegovan-data/chemprot/predictions_chemprot-bert-f1-2022-05-22-23-18-21-560/prediction_chemprot_test.json"
s3_test=[
"s3://aegovan-data/counterfactual_yelp/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040817/yelp_test.csv.json" # .25
, "s3://aegovan-data/counterfactual_amazon/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040817/amazon_test.csv.json" # .25
, "s3://aegovan-data/counterfactual_semeval/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040913/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json"  # .25
, "s3://aegovan-data/counterfactual_yelp/predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367/2023040911/yelp_test.csv.json" # .1,
, "s3://aegovan-data/counterfactual_semeval/predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367/2023040912/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json", #.1
 ]



In [2]:
import sys, os, pandas as pd
sys.path.append("../src")

import utils

In [3]:
from utils.s3_utils import download_file as download_s3_file


In [4]:
local_temp ="./temp"

In [5]:
local_prediction_dir = os.path.join(local_temp, "prediction")
os.makedirs(local_prediction_dir, exist_ok=True)

def load(s3_test):
    result = []
    for uri in s3_test:
        local_test_file  = download_s3_file( uri, local_prediction_dir)
        df_test = pd.read_json(local_test_file)
        df_test["uri"] = uri
        result.append(df_test)
    
    return result

result = load(s3_test)

In [6]:
result[0].head()

,Sentiment,Text,Negative,Positive,prediction,confidence,uri
0,Positive,"Contrary to other reviews, I have zero complai...",0.020631,0.979369,Positive,0.979369,s3://aegovan-data/counterfactual_yelp/predicti...
1,Negative,Last summer I had an appointment to get new ti...,0.276431,0.723569,Positive,0.723569,s3://aegovan-data/counterfactual_yelp/predicti...
2,Positive,"Friendly staff, same starbucks fair you get an...",0.060069,0.939931,Positive,0.939931,s3://aegovan-data/counterfactual_yelp/predicti...
3,Negative,The food is good. Unfortunately the service is...,0.992037,0.007963,Negative,0.992037,s3://aegovan-data/counterfactual_yelp/predicti...
4,Positive,Even when we didn't have a car Filene's Baseme...,0.014551,0.985449,Positive,0.985449,s3://aegovan-data/counterfactual_yelp/predicti...


In [10]:
from sklearn.metrics import accuracy_score
def consolidate_result(df_list): 
    result = []
    for df in df_list:

        score = accuracy_score(df["Sentiment"], df["prediction"])
        result.append({
            "Score" : round(score*100, 1),
            "Uri" : df["uri"].iloc[0],
            "TestSize": len(df),
            "Dataset" : df["uri"].iloc[0].split("/")[3]
        })
    return pd.DataFrame(result).sort_values(by=["Dataset", "Uri"])

consolidate_result(result)

,Score,Uri,TestSize,Dataset
1,89.9,s3://aegovan-data/counterfactual_amazon/predic...,400000,counterfactual_amazon
2,78.3,s3://aegovan-data/counterfactual_semeval/predi...,6069,counterfactual_semeval
4,62.7,s3://aegovan-data/counterfactual_semeval/predi...,6069,counterfactual_semeval
0,90.2,s3://aegovan-data/counterfactual_yelp/predicti...,37986,counterfactual_yelp
3,88.3,s3://aegovan-data/counterfactual_yelp/predicti...,37986,counterfactual_yelp
